In [80]:
import numpy as np
from neurons import poisson, LIF
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.palettes import mpl
output_notebook()

Loading BokehJS ...

In [95]:
layer0 = []
layer0_eval = []
layer1 = []
layer1_eval = []
layer2 = []
layer2_eval = []

for i in range(2):
    layer0.append(poisson())
for i in range(10):
    layer1.append(LIF())
for i in range(2):
    layer2.append(LIF())

w01 = np.random.rand(2, 10)
w12 = np.random.rand(10, 2)

T = 50
dt = 0.5
time = np.arange(0, T+dt, dt)

for i in range(len(layer0)):
    layer0_eval.append(layer0[i].execute(1, time))

syn01 = []
for i in range(10):
    sum = 0
    for j in range(2):
       sum += layer0_eval[j]*w01[j, i]
    syn01.append(sum)
syn01 = np.array(syn01)

for i, t in enumerate(time):
    for j in range(len(layer1)):
        layer1[j].execute(t, syn01[j, i])

syn12 = []
for i in range(2):
    sum = 0
    for j in range(10):
        sum += np.array(layer1[j].out_array)*w12[j, i]
    syn12.append(sum)
syn12 = np.array(syn12)

for i, t in enumerate(time):
    for j in range(len(layer2)):
        layer2[j].execute(t, syn12[j, i])

In [98]:
p = figure(height=400)
color = mpl['Plasma'][3]
for i in range(len(layer2)):
    # p.vbar(x=time, top=layer2[i].out_array, color=color[i], legend_label='LIF {}'.format(i+1))
    p.line(x=time, y=layer2[i].pot_plot, color=color[i], legend_label='LIF {}'.format(i+1), line_width=1.5)
show(p)